In [1]:
# 코랩을 사용하지 않으면 이 셀의 코드를 주석 처리하세요.
!git clone https://github.com/rickiepark/nlp-with-transformers.git
%cd nlp-with-transformers
from install import *
install_requirements(chapter=2)

Cloning into 'nlp-with-transformers'...
remote: Enumerating objects: 653, done.
remote: Counting objects: 100% (36/36), done.
remote: Compressing objects: 100% (11/11), done.
remote: Total 653 (delta 32), reused 25 (delta 25), pack-reused 617 (from 2)
Receiving objects: 100% (653/653), 62.59 MiB | 9.75 MiB/s, done.
Resolving deltas: 100% (333/333), done.
Updating files: 100% (118/118), done.
/content/nlp-with-transformers
⏳ Installing base requirements ...
✅ Base requirements installed!
Using transformers v4.48.3
Using datasets v3.3.2
Using accelerate v1.3.0
Using sentencepiece v0.2.0
Using umap v0.5.7


##data loading & emotion encoding

In [3]:
# 허깅페이스 데이터셋을 사용하기
from huggingface_hub import list_datasets
from datasets import load_dataset
from datasets import ClassLabel

emotions = load_dataset("emotion")

from transformers import AutoTokenizer
# SetFit/emotion 데이터셋의 경우 수동으로 ClassLabel 객체를 만들어 label에 할당합니다.
emotions['train'].features['label'] = ClassLabel(num_classes=6, names=['sadness', 'joy', 'love', 'anger', 'fear', 'surprise'])
model_ckpt = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)

def tokenize(batch):
    return tokenizer(batch["text"], padding=True, truncation=True)

emotions_encoded = emotions.map(tokenize, batched=True, batch_size=None)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/9.05k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/1.03M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/127k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/129k [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Map:   0%|          | 0/16000 [00:00<?, ? examples/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

##text tokenizing

In [4]:
from transformers import AutoModel
import torch
from sklearn.metrics import accuracy_score, f1_score

text = "this is a test"
inputs = tokenizer(text, return_tensors="pt")
inputs['input_ids'].size()

torch.Size([1, 6])

## HF login

In [5]:
from huggingface_hub import notebook_login

notebook_login()

## model

In [ ]:
from transformers import AutoModelForSequenceClassification

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
num_labels = 6
model_ckpt = "distilbert-base-uncased"

# distilbert-base-uncased가 바디이고 AutoModelForSequenceClassification가 헤드이다.
# num_label이 6이므로 6개의 감정 클래스를 분류하는 헤드가 추가된 것이다.
model = (AutoModelForSequenceClassification
         .from_pretrained(model_ckpt, num_labels=num_labels)
         .to(device))

## Learning

In [8]:
from transformers import Trainer, TrainingArguments

batch_size = 64
logging_steps = len(emotions_encoded["train"]) // batch_size
model_name = f"{model_ckpt}-finetuned-emotion"
training_args = TrainingArguments(output_dir=model_name,
                                  num_train_epochs=2,
                                  learning_rate=2e-5,
                                  per_device_train_batch_size=batch_size,
                                  per_device_eval_batch_size=batch_size,
                                  weight_decay=0.01,
                                  evaluation_strategy="epoch",
                                  disable_tqdm=False,
                                  logging_steps=logging_steps,
                                  push_to_hub=True,
                                  save_strategy="epoch",
                                  load_best_model_at_end=True,
                                  log_level="error",
                                  report_to="none")

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [9]:
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    f1 = f1_score(labels, preds, average="weighted")
    acc = accuracy_score(labels, preds)
    return {"accuracy": acc, "f1": f1}

trainer = Trainer(model=model, args=training_args,
                  compute_metrics=compute_metrics,
                  train_dataset=emotions_encoded["train"],
                  eval_dataset=emotions_encoded["validation"],
                  tokenizer=tokenizer)
trainer.train()

<ipython-input-9-7405cf54833b>:8: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(model=model, args=training_args,


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.852800,0.309132,0.912500,0.911724
2,0.252600,0.210840,0.930500,0.930384


TrainOutput(global_step=500, training_loss=0.552691795349121, metrics={'train_runtime': 251.0056, 'train_samples_per_second': 127.487, 'train_steps_per_second': 1.992, 'total_flos': 720342861696000.0, 'train_loss': 0.552691795349121, 'epoch': 2.0})

In [10]:
output = trainer.predict(emotions_encoded["validation"])
output.metrics

{'test_loss': 0.21083980798721313,
 'test_accuracy': 0.9305,
 'test_f1': 0.9303843178842454,
 'test_runtime': 3.9805,
 'test_samples_per_second': 502.446,
 'test_steps_per_second': 8.039}

In [11]:
import numpy as np
yy = np.argmax(output.predictions,axis=1)
yy

array([0, 0, 2, ..., 1, 1, 1])

##error analyze

In [12]:
from torch.nn.functional import cross_entropy

def forward_pass_with_label(batch):
    # 모든 입력 텐서를 모델과 같은 장치로 이동합니다.
    inputs = {k:v.to(device) for k,v in batch.items()
              if k in tokenizer.model_input_names}

    with torch.no_grad(): # 역전파 사용 x
        output = model(**inputs) # 입력 데이터를 모델에 전달
        pred_label = torch.argmax(output.logits, axis=-1) # 가장 높은 점수를 가진 클래스 선택
        loss = cross_entropy(output.logits, batch["label"].to(device), # loss 계산
                             reduction="none") # 평균을 내지 않고 개별 샘플의 손실을 반환

    return {"loss": loss.cpu().numpy(), # 결과를 CPU로 이동 및 numpy 배열로 변환
            "predicted_label": pred_label.cpu().numpy()}

In [13]:
# 데이터셋을 다시 파이토치 텐서로 변환
emotions_encoded.set_format("torch",
                            columns=["input_ids", "attention_mask", "label"])
# 손실 값을 계산
emotions_encoded["validation"] = emotions_encoded["validation"].map(
    forward_pass_with_label, batched=True, batch_size=16)

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

##int -> str 변환

In [16]:
def label_int2str(row):
    return emotions["train"].features["label"].int2str(row)

emotions_encoded.set_format("pandas")
cols = ["text", "label", "predicted_label", "loss"]
df_test = emotions_encoded["validation"][:][cols]
df_test["label"] = df_test["label"].apply(label_int2str)
df_test["predicted_label"] = (df_test["predicted_label"]
                              .apply(label_int2str))

In [40]:
df_test.sort_values("loss", ascending=False).head(10)

,text,label,predicted_label,loss
1801,i feel that he was being overshadowed by the s...,love,sadness,5.720547
882,i feel badly about reneging on my commitment t...,love,sadness,5.627258
1963,i called myself pro life and voted for perry w...,joy,sadness,5.383556
1950,i as representative of everything thats wrong ...,surprise,sadness,5.247468
1870,i guess i feel betrayed because i admired him ...,joy,sadness,5.125189
1500,i guess we would naturally feel a sense of lon...,anger,sadness,4.969193
1274,i am going to several holiday parties and i ca...,joy,sadness,4.924722
318,i felt ashamed of these feelings and was scare...,fear,sadness,4.550430
765,i feel super awkward and out of place right now,joy,sadness,4.513003
465,i would eventually go in to these stores but i...,joy,fear,4.362962


In [41]:
df_test.sort_values("loss", ascending=True).head(10)

,text,label,predicted_label,loss
131,i feel like a graph doesnt show the data accur...,joy,joy,0.016289
1873,i feel practically virtuous this month i have ...,joy,joy,0.016372
443,i feel i have to do its my creative calling my...,joy,joy,0.016989
1263,i feel this way about blake lively,joy,joy,0.017011
1418,i am a big believer in the phrase that some pe...,joy,joy,0.017044
632,i feel he is an terrific really worth bet,joy,joy,0.017154
1147,i type i feel bouncy and excited to get out my...,joy,joy,0.017277
620,i shall move right along to the post interview...,joy,joy,0.017410
1208,i feel brave again tubing were even going to t...,joy,joy,0.017619
578,i got to christmas feeling positive about the ...,joy,joy,0.017639


##save model & publish

In [42]:
trainer.push_to_hub(commit_message="Training completed!")

CommitInfo(commit_url='https://huggingface.co/SangJinCha/distilbert-base-uncased-finetuned-emotion/commit/426fb4eb81af4054523720114488861e4b621f99', commit_message='Training completed!', commit_description='', oid='426fb4eb81af4054523720114488861e4b621f99', pr_url=None, repo_url=RepoUrl('https://huggingface.co/SangJinCha/distilbert-base-uncased-finetuned-emotion', endpoint='https://huggingface.co', repo_type='model', repo_id='SangJinCha/distilbert-base-uncased-finetuned-emotion'), pr_revision=None, pr_num=None)

In [43]:
from transformers import pipeline

model_id = "SangJinCha/distilbert-base-uncased-finetuned-emotion"
classifier = pipeline("text-classification", model=model_id)

config.json:   0%|          | 0.00/883 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.23k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]